In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from pandas_datareader import data
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
import scipy.optimize as opt
import matplotlib.pyplot as plt

In [2]:
class stock(object):
    def __init__(self,symbol,from_date,to_date):
        self.raw_data = data.DataReader(symbol,'yahoo',from_date,to_date)
        self.ret = self.raw_data['Close'].pct_change()[1:]

In [3]:
from_date = '2016-01-01'
to_date = '2016-03-01'
symbols = ['AAPL','GOOG','AMZN','FB','MSFT']
bench = data.DataReader('SPY','yahoo',from_date,to_date)['Close'].pct_change()[1:]
stocks = {}
for symbol in symbols:
    stocks[symbol] = stock(symbol=symbol,from_date=from_date,to_date=to_date)

In [4]:
stock_prices = {}
for symbol in stocks:
    stock_prices[symbol] = stocks[symbol].ret
df = pd.DataFrame(stock_prices)
df = df.dropna()
print df.mean()
print df.cov()

AAPL   -0.000949
AMZN   -0.001937
FB      0.002384
GOOG   -0.000619
MSFT   -0.000846
dtype: float64
          AAPL      AMZN        FB      GOOG      MSFT
AAPL  0.000513  0.000322  0.000374  0.000236  0.000361
AMZN  0.000322  0.001037  0.000787  0.000405  0.000317
FB    0.000374  0.000787  0.001166  0.000531  0.000455
GOOG  0.000236  0.000405  0.000531  0.000386  0.000276
MSFT  0.000361  0.000317  0.000455  0.000276  0.000440


In [10]:
class portfolio(object):
    def __init__(self,data):
        self._data = data
        self._n = len(data.columns.values)
        self._w = np.matrix([np.ones(self._n)/self._n])
        self._r = np.matrix([data.mean().values]).T
        self._c = np.matrix(data.cov().values)
    
    def sharpe_ratio(self,w,r,c,rf):
        _w = np.matrix(w)
        _c = np.matrix(c)
        _r = np.matrix(r)
        return ((_w*_r-rf)[0,0])/(np.sqrt(_w*_c*_w.T)[0,0])
    
    def fitness(self,w,r,c,rf=0.0):
        return 1/self.sharpe_ratio(w,r,c,rf)
    
    def fitness_by_target_ret(self,w,r,c,rf=0.0):
        _w = np.matrix(w)
        _c = np.matrix(c)
        _r = np.matrix(r)
        return np.sqrt(_w*_c*_w.T)[0,0] + 100*np.abs((_w*_r)[0,0]-rf)
    
    def show_result(self,x):
        _w = np.matrix(x)
        print 'weight      : ',_w
        print 'sharp ratio : ',self.sharpe_ratio(_w,self._r,self._c,rf=0.0)
        print 'return      : ', (_w * self._r)[0,0]
        print 'variance    : ', (_w * self._c * _w.T)[0,0]
    
    def optimize(self):
        rf = 0.001
        b = [ (0.,1.) for i in range(self._n)]
        c = ({'type':'eq', 'fun': lambda w: sum(w)-1. })
        result = opt.minimize(self.fitness, self._w, 
                              args=(self._r,self._c,rf) ,method='SLSQP',constraints=c,bounds=b)
        self.show_result(result.x)
        
    def optimize_by_target_ret(self,rf=0.0):
        b = [ (0.,1.) for i in range(self._n)]
        c = ({'type':'eq', 'fun': lambda w: sum(w)-1. })
        result = opt.minimize(self.fitness_by_target_ret, self._w, 
                              args=(self._r,self._c,rf) ,method='SLSQP',constraints=c,bounds=b)
        self.show_result(result.x)
    
        
p = portfolio(df)
p.optimize()
p.optimize_by_target_ret(0.000999684361617)

weight      :  [[ 0.09967216  0.09967216  0.60131134  0.09967216  0.09967216]]
sharp ratio :  0.0367345613268
return      :  0.000999684361617
variance    :  0.000740586993838
weight      :  [[  1.32476371e-01   8.94466792e-18   5.64036451e-01   1.66222423e-01
    1.37264754e-01]]
sharp ratio :  0.0388871772988
return      :  0.000999683122079
variance    :  0.000660863672308
